In [1]:
from requests import Request, Session
from bs4 import BeautifulSoup
import numpy
import pandas as pd

In [2]:
#set the search term here
search_term = 'iphone'

In [3]:
page = 1
page_limit = 0
soups = []

In [4]:
def urlfier(url):
    return ''.join(url)

def extractText(info):
    return BeautifulSoup(str(info), "html.parser").find(text=True)

In [5]:
# Headers Setup
headers = {    
    #setting as default browser.
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    
    #some configs i thought was important (i don't know yet)
    "authority": "www.amazon.com.br",
    "scheme": "https",
    "origin": "https://www.amazon.com.br",
    
    "accept": "text/html,*/*",
    "accept-language": "en-US,en;q=0.9,ja-JP;q=0.8,ja;q=0.7,pt-BR;q=0.6,pt;q=0.5",
    "content-type": "application/json",
    "downlink": "50",
    "ect": "4g",
    "rtt": "100",
    "sec-ch-ua": "\"Google Chrome\";v=\"87\", \" Not;A Brand\";v=\"99\", \"Chromium\";v=\"87\"",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-amazon-rush-fingerprints": "AmazonRushAssetLoader:82DC4FCDBC1BD29C851447C5E56300ACEBC3484E|AmazonRushFramework:C63458EFD5C6F3B9DB4C55251BDF45CAB84ACA9E|AmazonRushRouter:C88B480F313C8089B887E17289DB6E9437386461|SearchAssets:68AEE251205924E94A40052B6F4BBA8540827F9F|DynamicImageLoader:72044AC19F5D25484C3A6A210715639EB294450A|SearchPartnerAssets:C3828FDE58240400D60660B659C2BB69C3AF2C8C",
    "x-amazon-s-fallback-url": "",
    "x-amazon-s-mismatch-behavior": "ALLOW",
    "x-amazon-s-swrs-version": "0265482AAA9506275F3AC0E0B1B2250D",
    "x-requested-with": "XMLHttpRequest"
}

headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
 'authority': 'www.amazon.com.br',
 'scheme': 'https',
 'origin': 'https://www.amazon.com.br',
 'accept': 'text/html,*/*',
 'accept-language': 'en-US,en;q=0.9,ja-JP;q=0.8,ja;q=0.7,pt-BR;q=0.6,pt;q=0.5',
 'content-type': 'application/json',
 'downlink': '50',
 'ect': '4g',
 'rtt': '100',
 'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
 'sec-ch-ua-mobile': '?0',
 'sec-fetch-dest': 'empty',
 'sec-fetch-mode': 'cors',
 'sec-fetch-site': 'same-origin',
 'x-amazon-rush-fingerprints': 'AmazonRushAssetLoader:82DC4FCDBC1BD29C851447C5E56300ACEBC3484E|AmazonRushFramework:C63458EFD5C6F3B9DB4C55251BDF45CAB84ACA9E|AmazonRushRouter:C88B480F313C8089B887E17289DB6E9437386461|SearchAssets:68AEE251205924E94A40052B6F4BBA8540827F9F|DynamicImageLoader:72044AC19F5D25484C3A6A210715639EB294450A|SearchPartnerAssets:C3828FDE58240400D60660B659C2BB69C

In [6]:
URL = ['https://www.amazon.com.br/s?k=', search_term,'&page=', str(page)]
prepped = Request('GET',
                  urlfier(URL),
                  headers=headers,
                  #params=payload
                 ).prepare()
prepped.url, urlfier(URL)

('https://www.amazon.com.br/s?k=iphone&page=1',
 'https://www.amazon.com.br/s?k=iphone&page=1')

In [7]:
s = Session() # Session Setup
resp = s.send(prepped) #request

soups.append(BeautifulSoup(resp.content, 'html.parser')) #appending first request to "soups list"

resp, len(soups)

(<Response [200]>, 1)

In [8]:
#getting how much pages has the search
aux = soups[0].select('ul.a-pagination li[aria-disabled=true]')
if len(aux) == 0:
    page_limit = 0
else:
    page_limit = aux[0].get_text()

page_limit

'7'

In [9]:
if int(page_limit) != 0:
    i = 2
    while i <= int(page_limit):
        resp = None
        
        URL[3] = str(i)
        prepped.url = urlfier(URL)
        
        resp = s.send(prepped)
        soups.append(BeautifulSoup(resp.content, 'html.parser'))
        
        print(f'Requesting page {len(soups)}')
        
        i = i + 1

print(f'finished with {len(soups)} requests.')

Requesting page 2
Requesting page 3
Requesting page 4
Requesting page 5
Requesting page 6
Requesting page 7


('total ', 7)

In [11]:
#get all products of all pages
products = []
for page in soups:
    products += page.select('div [data-index][data-uuid]')

print(f'{len(products)} products found.')

306 products found.


In [24]:
rep = []
ext = []

#extract all products infos
for item in products:
    aux1 = []
    aux2 = []
    
    #get name of product
    name = ''
    name = extractText(item.select('h2 span')[0])
  
    #get product price
    price = item.select('span.a-offscreen')

    #get stock of product
    stock = item.select('div span[aria-label="Não disponível."] span[dir]')

    if len(price) == 0:
        if len(stock) == 0:
            price = 'Preço à tratar com vendedores.'
        else:
            price = 'Sem Estoque.'
    else:
        price = extractText(price[0])

    aux1.append([name, price]) #appending info into (auxiliary var) 

    stars = item.select('span.a-icon-alt') #get stars of product
    if len(stars) == 0:
        stars = 'Sem avaliação'
    else:
        stars = extractText(stars[0])

    rates = item.select('span[aria-label] a span.a-size-base') #rates in a product
    if len(rates) == 0:
        rates = 'Sem avaliação'
    else:
        rates = extractText(rates[0])

    #link of an product
    link = 'https://www.amazon.com.br' + item.select('h2 a')[0].get('href')

    #appending info into (auxiliary var)
    aux2.append([link, stars, rates])

    rep.append(aux1[0])
    ext.append(aux2[0])
    
len(rep), len(ext)

(306, 306)

In [28]:
rep[15]

['Cabo Lightning Certificado Apple MFi, Anker Powerline II, 1.8 metros, 12x mais resistente, Preto',
 'R$89,90']

In [29]:
products_main = numpy.asarray(rep)
products_sec = numpy.asarray(ext)

dfm = pd.DataFrame(products_main)
dfe = pd.DataFrame(products_sec)

print(dfm.to_string())

                                                                                                                                                                                                                                                       0                               1
0                                                                                                                                                                                                   Iphone Xr Apple Preto, 64gb Desbloqueado - Mry42br/a  Preço à tratar com vendedores.
1                                                                                                                                                                                                               Novo Iphone Xr 64Gb Branco Ios 12 4G ...  Preço à tratar com vendedores.
2                                                                                                                                                            

In [30]:
dfm.to_csv(search_term + '_main.csv')
dfe.to_csv(search_term + '_extra.csv')